In [1]:
import numpy as np
import pandas as pd
import model.preprocess_games as ppg
from pandas import DataFrame
pd.options.display.max_columns = None
pd.set_option("display.max_colwidth", -1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", 101)

/home/nathaniel/anaconda3/envs/nba-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
DATA_PATH = 'data'

SEASONS_PROCESSED_DS = f"{DATA_PATH}/seasons.processed.feather"

TEAMS_DS = f"{DATA_PATH}/teams.processed.feather"
TEAMS_PROCESSED_DS = f"{DATA_PATH}/teams.processed.feather"

GAMES_DS = f"{DATA_PATH}/games.csv"
GAMES_PROCESSED_DS = f"{DATA_PATH}/games.processed.feather"

In [3]:
games = pd.read_csv(GAMES_DS, parse_dates=["GAME_DATE_EST"], usecols=["GAME_ID",'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
       'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away',
       'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS']
                    , infer_datetime_format=True, index_col="GAME_ID")
games = games.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])
teams = pd.read_feather(TEAMS_PROCESSED_DS)
seasons = pd.read_feather(SEASONS_PROCESSED_DS)
games_matchup = pd.read_feather(GAMES_PROCESSED_DS)
games_matchup.set_index(["GAME_ID"])

,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HT,HT_RANK,HT_CLASS,HT_HW,HT_HL,HT_VW,HT_VL,HT_LAST10_W,HT_LAST10_L,HT_LAST10_MATCHUP_W,HT_LAST10_MATCHUP_L,HT_OVERALL_OFF_POINTS,HT_OVERALL_DEF_POINTS,HT_OVERALL_OFF_FG,HT_OVERALL_DEF_FG,HT_OVERALL_OFF_3P,HT_OVERALL_DEF_3P,HT_OVERALL_OFF_FT,HT_OVERALL_DEF_FT,HT_OVERALL_OFF_REB,HT_OVERALL_DEF_REB,HT_AWAY_POINTS,HT_AWAY_FG,HT_AWAY_3P,HT_AWAY_FT,HT_AWAY_REB,VT,VT_RANK,VT_CLASS,VT_HW,VT_HL,VT_VW,VT_VL,VT_LAST10_W,VT_LAST10_L,VT_LAST10_MATCHUP_W,VT_LAST10_MATCHUP_L,VT_OVERALL_OFF_POINTS,VT_OVERALL_DEF_POINTS,VT_OVERALL_OFF_FG,VT_OVERALL_DEF_FG,VT_OVERALL_OFF_3P,VT_OVERALL_DEF_3P,VT_OVERALL_OFF_FT,VT_OVERALL_DEF_FT,VT_OVERALL_OFF_REB,VT_OVERALL_DEF_REB,VT_AWAY_POINTS,VT_AWAY_FG,VT_AWAY_3P,VT_AWAY_FT,VT_AWAY_REB
GAME_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21801229,2019-04-10,Final,1610612746,1610612762,2018,143.0,0.509,0.767,0.429,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1,1610612746,None,None,248,112,186,175,6,4,4,6,105.594,102.372,0.470,0.449,0.361,0.350,0.726,0.759,42.839,42.852,104.172,0.463,0.362,0.723,42.139,1610612762,None,None,223,138,147,213,8,2,6,4,100.128,99.331,0.458,0.454,0.350,0.363,0.752,0.762,42.947,40.990,98.817,0.451,0.346,0.752,42.197
21801228,2019-04-10,Final,1610612743,1610612750,2018,99.0,0.448,0.647,0.303,23.0,53.0,95.0,0.429,0.667,0.406,24.0,41.0,1,1610612743,None,None,236,124,147,214,4,6,5,5,106.488,105.436,0.462,0.462,0.352,0.358,0.749,0.756,44.706,42.234,104.310,0.455,0.346,0.747,44.017,1610612750,None,None,167,194,106,254,4,6,5,5,103.398,105.759,0.452,0.472,0.340,0.365,0.781,0.763,42.933,42.824,102.950,0.451,0.337,0.785,42.397
21801227,2019-04-10,Final,1610612759,1610612742,2018,105.0,0.466,0.833,0.333,22.0,53.0,94.0,0.407,0.750,0.297,27.0,42.0,1,1610612759,None,None,299,61,209,152,5,5,8,2,104.706,98.558,0.476,0.449,0.383,0.350,0.785,0.762,43.301,41.986,102.828,0.465,0.365,0.784,42.906,1610612742,None,None,212,149,153,207,5,5,2,8,102.219,101.958,0.456,0.456,0.356,0.353,0.776,0.758,41.926,44.452,100.544,0.451,0.356,0.772,41.053
21801226,2019-04-10,Final,1610612749,1610612760,2018,116.0,0.430,0.714,0.326,27.0,53.0,127.0,0.485,0.615,0.426,40.0,53.0,0,1610612749,None,None,197,163,142,219,7,3,3,7,101.178,101.809,0.457,0.453,0.354,0.355,0.760,0.763,42.429,43.613,100.158,0.453,0.349,0.753,42.186,1610612760,None,None,267,94,197,163,6,4,7,3,107.044,102.362,0.464,0.446,0.352,0.352,0.774,0.758,45.669,41.648,105.281,0.454,0.340,0.773,45.628
21801225,2019-04-10,Final,1610612763,1610612744,2018,132.0,0.490,0.833,0.457,25.0,49.0,117.0,0.500,0.857,0.433,32.0,39.0,1,1610612763,None,None,232,128,155,206,3,7,3,7,98.387,98.139,0.452,0.450,0.339,0.355,0.768,0.761,41.922,41.574,96.961,0.449,0.336,0.764,41.080,1610612744,None,None,266,95,213,147,8,2,7,3,108.949,103.748,0.478,0.443,0.391,0.343,0.779,0.761,44.004,43.886,107.528,0.469,0.384,0.774,43.608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21600005,2016-10-26,Final,1610612754,1610612742,2016,130.0,0.505,0.765,0.526,30.0,52.0,121.0,0.433,0.722,0.375,26.0,49.0,1,1610612754,None,None,161,77,106,131,7,3,7,3,98.099,96.036,0.444,0.435,0.353,0.342,0.768,0.753,44.526,42.488,96.127,0.437,0.353,0.770,43.346,1610612742,None,None,152,86,123,115,7,3,3,7,101.782,100.162,0.461,0.451,0.358,0.349,0.780,0.751,42.036,44.170,99.992,0.455,0.359,0.779,41.420
21600004,2016-10-26,Final,1610612753,1610612748,2016,96.0,0.382,0.786,0.333,20.0,45.0,108.0,0.485,0.625,0.250,27.0,52.0,0,1610612753,None,None,117,121,75,163,6,4,2,8,97.055,99.410,0.451,0.456,0.351,0.355,0.732,0.753,42.597,42.626,95.265,0.444,0.343,0.730,41.798,1

In [4]:
games.info()
games.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23195 entries, 10300001 to 21900901
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_DATE_EST     23195 non-null  datetime64[ns]
 1   GAME_STATUS_TEXT  23195 non-null  object        
 2   HOME_TEAM_ID      23195 non-null  int64         
 3   VISITOR_TEAM_ID   23195 non-null  int64         
 4   SEASON            23195 non-null  int64         
 5   PTS_home          23096 non-null  float64       
 6   FG_PCT_home       23096 non-null  float64       
 7   FT_PCT_home       23096 non-null  float64       
 8   FG3_PCT_home      23096 non-null  float64       
 9   AST_home          23096 non-null  float64       
 10  REB_home          23096 non-null  float64       
 11  PTS_away          23096 non-null  float64       
 12  FG_PCT_away       23096 non-null  float64       
 13  FT_PCT_away       23096 non-null  float64       
 14  FG3_PCT_away

Index(['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home',
       'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away',
       'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [5]:
df = games_matchup[games_matchup.SEASON==2018]
row = df.tail(1).iloc[0, :]
display(row)
type(row)

GAME_ID                  21800001           
GAME_DATE_EST            2018-10-16 00:00:00
GAME_STATUS_TEXT         Final              
HOME_TEAM_ID             1610612738         
VISITOR_TEAM_ID          1610612755         
SEASON                   2018               
PTS_home                 105                
FG_PCT_home              0.433              
FT_PCT_home              0.714              
FG3_PCT_home             0.297              
AST_home                 21                 
REB_home                 55                 
PTS_away                 87                 
FG_PCT_away              0.391              
FT_PCT_away              0.609              
FG3_PCT_away             0.192              
AST_away                 18                 
REB_away                 47                 
HOME_TEAM_WINS           1                  
HT                       1610612738         
HT_RANK                  None               
HT_CLASS                 None               
HT_HW     

pandas.core.series.Series

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1229 entries, 0 to 1228
Data columns (total 71 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GAME_ID                1229 non-null   int64         
 1   GAME_DATE_EST          1229 non-null   datetime64[ns]
 2   GAME_STATUS_TEXT       1229 non-null   object        
 3   HOME_TEAM_ID           1229 non-null   int64         
 4   VISITOR_TEAM_ID        1229 non-null   int64         
 5   SEASON                 1229 non-null   int64         
 6   PTS_home               1229 non-null   float64       
 7   FG_PCT_home            1229 non-null   float64       
 8   FT_PCT_home            1229 non-null   float64       
 9   FG3_PCT_home           1229 non-null   float64       
 10  AST_home               1229 non-null   float64       
 11  REB_home               1229 non-null   float64       
 12  PTS_away               1229 non-null   float64       
 13  FG_

In [7]:
teams[teams.TEAM_ID == 1610612738]

,TEAM_ID,NAME,NICKNAME,CITY
1,1610612738,BOS,Celtics,Boston


In [ ]:
teams_df = teams.drop(columns=["NICKNAME", "CITY"])
df_merged = df.merge(teams_df, left_on='HOME_TEAM_ID', right_on='TEAM_ID', suffixes=['_games', '_teams'])
df_merged = df_merged.drop(columns=["TEAM_ID"])
df_merged = df_merged.rename(columns={"NAME": "HOME_TEAM"})
df_merged

In [19]:
teams_df = teams.drop(columns=["NICKNAME", "CITY"])
df_merged = df.merge(teams_df, left_on='HOME_TEAM_ID', right_on='TEAM_ID', suffixes=['_games', '_teams'])
df_merged = df_merged.drop(columns=["TEAM_ID"])
df_merged = df_merged.rename(columns={"NAME": "HOME_TEAM"})
df_merged

,GAME_ID,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HT,HT_RANK,HT_CLASS,HT_HW,HT_HL,HT_VW,HT_VL,HT_LAST10_W,HT_LAST10_L,HT_LAST10_MATCHUP_W,HT_LAST10_MATCHUP_L,HT_OVERALL_OFF_POINTS,HT_OVERALL_DEF_POINTS,HT_OVERALL_OFF_FG,HT_OVERALL_DEF_FG,HT_OVERALL_OFF_3P,HT_OVERALL_DEF_3P,HT_OVERALL_OFF_FT,HT_OVERALL_DEF_FT,HT_OVERALL_OFF_REB,HT_OVERALL_DEF_REB,HT_AWAY_POINTS,HT_AWAY_FG,HT_AWAY_3P,HT_AWAY_FT,HT_AWAY_REB,VT,VT_RANK,VT_CLASS,VT_HW,VT_HL,VT_VW,VT_VL,VT_LAST10_W,VT_LAST10_L,VT_LAST10_MATCHUP_W,VT_LAST10_MATCHUP_L,VT_OVERALL_OFF_POINTS,VT_OVERALL_DEF_POINTS,VT_OVERALL_OFF_FG,VT_OVERALL_DEF_FG,VT_OVERALL_OFF_3P,VT_OVERALL_DEF_3P,VT_OVERALL_OFF_FT,VT_OVERALL_DEF_FT,VT_OVERALL_OFF_REB,VT_OVERALL_DEF_REB,VT_AWAY_POINTS,VT_AWAY_FG,VT_AWAY_3P,VT_AWAY_FT,VT_AWAY_REB,HOME_TEAM
0,21801229,2019-04-10,Final,1610612746,1610612762,2018,143.0,0.509,0.767,0.429,34.0,52.0,137.0,0.443,0.879,0.400,31.0,57.0,1,1610612746,None,None,248,112,186,175,6,4,4,6,105.594,102.372,0.47,0.449,0.361,0.350,0.726,0.759,42.839,42.852,104.172,0.463,0.362,0.723,42.139,1610612762,None,None,223,138,147,213,8,2,6,4,100.128,99.331,0.458,0.454,0.350,0.363,0.752,0.762,42.947,40.990,98.817,0.451,0.346,0.752,42.197,LAC
1,21801192,2019-04-05,Final,1610612746,1610612747,2018,117.0,0.440,0.871,0.345,29.0,45.0,122.0,0.448,0.815,0.438,28.0,55.0,0,1610612746,None,None,248,111,186,174,8,2,8,2,105.580,102.305,0.47,0.449,0.360,0.350,0.726,0.758,42.841,42.828,104.172,0.463,0.361,0.723,42.147,1610612747,None,None,181,178,123,237,4,6,2,8,102.764,105.558,0.452,0.460,0.342,0.351,0.741,0.750,44.302,44.631,102.150,0.446,0.348,0.746,43.536,LAC
2,21801176,2019-04-03,Final,1610612746,1610612745,2018,103.0,0.463,0.613,0.308,27.0,38.0,135.0,0.534,0.793,0.474,22.0,48.0,0,1610612746,None,None,248,110,186,174,8,2,6,4,105.584,102.259,0.47,0.449,0.361,0.349,0.726,0.758,42.848,42.820,104.172,0.463,0.361,0.723,42.147,1610612745,None,None,255,104,188,171,8,2,4,6,107.844,104.196,0.457,0.458,0.357,0.345,0.756,0.765,43.379,43.097,106.577,0.455,0.357,0.757,42.465,LAC
3,21801151,2019-03-31,Final,1610612746,1610612763,2018,113.0,0.480,0.895,0.259,23.0,47.0,96.0,0.376,0.828,0.242,22.0,34.0,1,1610612746,None,None,247,110,186,174,8,2,6,4,105.573,102.268,0.47,0.449,0.361,0.350,0.726,0.758,42.842,42.833,104.172,0.463,0.361,0.723,42.147,1610612763,None,None,232,127,154,203,5,5,4,6,98.338,98.028,0.452,0.450,0.339,0.355,0.768,0.761,41.919,41.538,96.927,0.449,0.336,0.763,41.095,LAC
4,21801136,2019-03-30,Final,1610612746,1610612739,2018,132.0,0.624,0.688,0.455,33.0,44.0,108.0,0.446,0.821,0.130,18.0,36.0,1,1610612746,None,None,246,110,186,174,8,2,5,5,105.536,102.260,0.47,0.449,0.361,0.350,0.726,0.758,42.841,42.842,104.172,0.463,0.361,0.723,42.147,1610612739,None,None,193,166,134,223,3,7,5,5,101.992,103.888,0.450,0.467,0.358,0.366,0.753,0.755,42.641,42.806,101.056,0.446,0.353,0.756,41.692,LAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,21800117,2018-11-02,Final,1610612753,1610612746,2018,95.0,0.444,0.579,0.235,20.0,40.0,120.0,0.477,0.758,0.500,22.0,50.0,0,1610612753,None,None,151,173,97,226,3,7,1,9,98.406,101.692,0.45,0.459,0.347,0.357,0.738,0.759,42.586,43.491,97.142,0.445,0.342,0.739,41.653,1610612746,None,None,226,98,165,159,4,6,9,1,104.562,101.032,0.470,0.448,0.357,0.350,0.719,0.757,42.560,42.537,103.281,0.463,0.359,0.716,41.907,ORL
1225,21800098,2018-10-30,Final,1610612753,1610612758,2018,99.0,0.396,0.727,0.256,28.0,48.0,107.0,0.488,0.654,0.364,22.0,51.0,0,1610612753,None,None,151,172,97,226,3,7,4,6,98.406,101.684,0.45,0.459,0.347,0.357,0.738,0.759,42.577,43.480,97.142,0.445,0.342,0.739,41.653,16106127